In [13]:
import spacy
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [14]:
df = pd.read_csv('_6_2_Train.csv')
df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [15]:
y = df.target
X_raw = df['excerpt']

print(X_raw.shape)
print(y.shape)
X_raw.head()


(2834,)
(2834,)


0    When the young people returned to the ballroom...
1    All through dinner time, Mrs. Fayre was somewh...
2    As Roger had predicted, the snow departed as q...
3    And outside before the palace a great garden w...
4    Once upon a time there were Three Bears who li...
Name: excerpt, dtype: object

In [21]:
def preprocess(X_raw):
    nlp = spacy.load('en_core_web_sm') 
    docs = [nlp(exc) for exc in X_raw]

    # helper lists of lists to derive features below 
    doc_tok_lens = [[len(tok.text) for tok in doc] for doc in docs]
    doc_tok_lens_sq = [[tok_len**2 for tok_len in lengths] for lengths in doc_tok_lens]

    # n_toks
    doc_n_toks = [len(doc) for doc in docs]
    # n_stop_toks
    doc_n_stop_toks = [sum([tok.is_stop for tok in doc]) for doc in docs]
    # n_chars
    doc_n_chars = [len(doc.text) for doc in docs]
    # tok_len_mean
    doc_tok_len_means = [sum(lens)/len(lens) for lens in doc_tok_lens]
    # tok_len_var
    doc_tok_len_vars = [sum(lens2)/len(lens2) - (sum(lens)/len(lens))**2 for lens2, lens in zip(doc_tok_lens_sq, doc_tok_lens)]
    # n_punct
    doc_all_punct = [sum([tok.pos_ == 'PUNCT' for tok in doc]) for doc in docs]
    # n_special_punct
    doc_nonstd_punct = [sum([tok.lemma_ in ['-', ','] for tok in doc]) for doc in docs]

    # see https://universaldependencies.org/docs/u/pos/ for pos used by spaCy
    pos_all = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']
    pos_dict = dict(zip(pos_all, list(range(len(pos_all)))))

    # pos_dist
    doc_pos_counts = [[0]*len(pos_all) for doc in docs]
    doc_pos_dist = [[0]*len(pos_all) for doc in docs]
    for i, doc in enumerate(docs):
        for tok in doc:
            if tok.pos_ not in pos_all:
                print("i = ", i, "   lemma = ", tok.lemma_)
            doc_pos_counts[i][pos_dict[tok.pos_]] += 1
        doc_pos_dist[i] = np.divide(doc_pos_counts[i], doc_n_toks[i])

    X = pd.DataFrame(doc_pos_dist, columns=['pos_' + pos.lower() for pos in pos_all])
    X['n_toks'] = doc_n_toks
    X['n_chars'] = doc_n_chars
    X['prop_stop_toks'] = np.divide(doc_n_stop_toks, doc_n_toks)
    X['tok_len_mean'] = doc_tok_len_means
    X['tok_len_var'] = doc_tok_len_vars
    X['prop_punct'] = np.divide(doc_all_punct, doc_n_chars)
    X['prop_nonstd_punct'] = np.divide(doc_nonstd_punct, doc_all_punct)

    return X

In [22]:
X = preprocess(X_raw)

     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 MB 435.7 kB/s eta 0:22:29
     -------------------------------------- 0.0/587.7 

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\VIVEK\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\VIVEK\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\VIVEK\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\VIVEK\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\VIVEK\anaconda3\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "C:\Users\VIVEK\anaconda3\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\VIVEK\anaconda3\lib\ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\VIVEK\anaconda3\lib

     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     -------------------------------------- 0.1/587.7 MB 525.1 kB/s eta 0:18:40
     -------------------------------------- 0.1/587.7 MB 726.2 kB/s eta 0:13:30
     -------------------------------------- 0.1/587.7 MB 787.7 kB/s eta 0:12:26
     -------------------------------------- 0.2/587.7 MB 875.2 kB/s eta 0:11:12
     -------------------------------------- 0.3/587.7 MB 983.9 kB/s eta 0:09:58
     ---------------------------------------- 0.4/587.7 MB 1.1 MB/s eta 0:08:33
     ---------------------------------------- 0.5/587.7 MB 1.2 MB/s eta 0:08:21
     ---------------------------------------- 0.5/587.7 MB 1.3 MB/s eta 0:07:46
     ---------------------------------------- 0.6/587.7 MB 1.3 MB/s eta 0:07:44
     ---------------------------------------- 0.7/587.7 MB 1.3 MB/s eta 0:07:29
     ---------------------------------------- 0.8/587.

In [23]:
X_train, X_val, y_train_actual, y_val_actual = train_test_split(X, y, test_size=0.15, random_state=1)

In [24]:
rf = RandomForestRegressor(n_estimators=100, random_state=1)
rf.fit(X_train, y_train_actual)

RandomForestRegressor(random_state=1)

In [25]:
y_train_predicted = rf.predict(X_train)
y_val_predicted = rf.predict(X_val)
train_mse = mean_squared_error(y_train_predicted, y_train_actual)
valid_mse = mean_squared_error(y_val_predicted, y_val_actual)
print(f"baseline model training MSE = {train_mse:.6f}")
print(f"baseline model validation MSE = {valid_mse:.6f}")

baseline model training MSE = 0.089607
baseline model validation MSE = 0.612715
